# Reverse Query Architecture Benchmark - Example Queries

This notebook demonstrates the performance differences between the 3 architectures with concrete examples from the benchmark.

**Key Findings:**
- **GraphRAG**: 100% recall, 0.09s avg latency
- **Format_B_Chunked**: 98.88% recall, 82.44s avg latency
- **Format_A**: 7.97% recall, 23.42s avg latency

## Setup

In [ ]:
import json
import time
from pathlib import Path
import sys

# Add parent directory to path
sys.path.append('..')

from src.architectures.rag_format_b import FormatBRAG
from src.architectures.rag_format_a import FormatARAG
from src.architectures.graphrag import GraphRAG

print("✅ Imports successful")

## Load Ground Truth and Sample Queries

In [ ]:
# Load ground truth
with open('../data/processed/neo4j_ground_truth.json', 'r') as f:
    ground_truth = json.load(f)

# Load benchmark sample
with open('../data/processed/benchmark_sample_20251103_091714.json', 'r') as f:
    sample_queries = json.load(f)

print(f"Loaded ground truth for {len(ground_truth)} side effects")
print(f"Loaded {len(sample_queries)} sample queries")

# Group by tier
tier_queries = {}
for q in sample_queries:
    tier = q['tier']
    if tier not in tier_queries:
        tier_queries[tier] = []
    tier_queries[tier].append(q)

for tier, queries in tier_queries.items():
    print(f"  {tier}: {len(queries)} queries")

## Initialize Architectures

In [ ]:
print("Initializing architectures...")

format_b = FormatBRAG()
print("✅ Format_B_Chunked initialized")

format_a = FormatARAG()
print("✅ Format_A initialized")

graphrag = GraphRAG()
print("✅ GraphRAG initialized")

## Helper Function for Evaluation

In [ ]:
def evaluate_query(side_effect, architecture, arch_name):
    """Run a single query and calculate metrics"""
    
    expected_drugs = set([d.lower() for d in ground_truth.get(side_effect, [])])
    
    # Time the query
    start = time.time()
    result = architecture.reverse_query(side_effect)
    latency = time.time() - start
    
    extracted_drugs = set([d.lower() for d in result.get('drugs', [])])
    
    # Calculate metrics
    tp = len(extracted_drugs & expected_drugs)
    fp = len(extracted_drugs - expected_drugs)
    fn = len(expected_drugs - extracted_drugs)
    
    recall = tp / len(expected_drugs) if expected_drugs else 0
    precision = tp / len(extracted_drugs) if extracted_drugs else 0
    
    return {
        'architecture': arch_name,
        'side_effect': side_effect,
        'expected_count': len(expected_drugs),
        'extracted_count': len(extracted_drugs),
        'recall': recall,
        'precision': precision,
        'latency': latency,
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn
    }

def print_results(results):
    """Pretty print results"""
    print(f"\n{'='*80}")
    print(f"Side Effect: {results['side_effect']}")
    print(f"Architecture: {results['architecture']}")
    print(f"{'='*80}")
    print(f"Expected drugs:  {results['expected_count']}")
    print(f"Extracted drugs: {results['extracted_count']}")
    print(f"\nMetrics:")
    print(f"  Recall:    {results['recall']*100:.2f}%")
    print(f"  Precision: {results['precision']*100:.2f}%")
    print(f"  Latency:   {results['latency']:.2f}s")
    print(f"\nConfusion:")
    print(f"  True Positives:  {results['true_positives']}")
    print(f"  False Positives: {results['false_positives']}")
    print(f"  False Negatives: {results['false_negatives']}")
    print(f"{'='*80}")

## Example 1: Large Tier Query (500+ drugs)

**Side Effect:** "nausea" (658 drugs in SIDER)

This is a challenging query with many drugs to extract.

In [ ]:
# Find a large tier example from our sample
large_example = [q for q in tier_queries.get('large', []) if q['side_effect'] == 'nausea'][0]
print(f"Testing: {large_example['side_effect']} (tier: {large_example['tier']})")
print(f"Expected drugs: {len(ground_truth[large_example['side_effect']])}")

### GraphRAG - Large Tier

In [ ]:
result_graphrag = evaluate_query(large_example['side_effect'], graphrag, 'GraphRAG')
print_results(result_graphrag)

### Format_B_Chunked - Large Tier

In [ ]:
result_format_b = evaluate_query(large_example['side_effect'], format_b, 'Format_B_Chunked')
print_results(result_format_b)

### Format_A - Large Tier

In [ ]:
result_format_a = evaluate_query(large_example['side_effect'], format_a, 'Format_A')
print_results(result_format_a)

### Comparison - Large Tier

In [ ]:
print("\n" + "="*80)
print(f"LARGE TIER COMPARISON: {large_example['side_effect']}")
print("="*80)
print(f"{'Architecture':<20} {'Recall':<12} {'Precision':<12} {'Latency':<12} {'Speedup'}")
print("-"*80)

baseline_latency = result_format_b['latency']
for res in [result_graphrag, result_format_b, result_format_a]:
    speedup = baseline_latency / res['latency']
    print(f"{res['architecture']:<20} {res['recall']*100:<11.2f}% {res['precision']*100:<11.2f}% {res['latency']:<11.2f}s {speedup:.1f}×")

print("="*80)
print(f"\n💡 GraphRAG is {baseline_latency/result_graphrag['latency']:.0f}× faster than Format_B!")

## Example 2: Medium Tier Query (100-499 drugs)

**Side Effect:** From medium tier sample

In [ ]:
# Pick first medium tier example
medium_example = tier_queries['medium'][0]
print(f"Testing: {medium_example['side_effect']} (tier: {medium_example['tier']})")
print(f"Expected drugs: {len(ground_truth[medium_example['side_effect']])}")

In [ ]:
# Run all three architectures
medium_graphrag = evaluate_query(medium_example['side_effect'], graphrag, 'GraphRAG')
medium_format_b = evaluate_query(medium_example['side_effect'], format_b, 'Format_B_Chunked')
medium_format_a = evaluate_query(medium_example['side_effect'], format_a, 'Format_A')

# Compare
print("\n" + "="*80)
print(f"MEDIUM TIER COMPARISON: {medium_example['side_effect']}")
print("="*80)
print(f"{'Architecture':<20} {'Recall':<12} {'Precision':<12} {'Latency':<12}")
print("-"*80)

for res in [medium_graphrag, medium_format_b, medium_format_a]:
    print(f"{res['architecture']:<20} {res['recall']*100:<11.2f}% {res['precision']*100:<11.2f}% {res['latency']:<11.2f}s")

print("="*80)

## Example 3: Small Tier Query (20-99 drugs)

In [ ]:
# Pick first small tier example
small_example = tier_queries['small'][0]
print(f"Testing: {small_example['side_effect']} (tier: {small_example['tier']})")
print(f"Expected drugs: {len(ground_truth[small_example['side_effect']])}")

In [ ]:
# Run all three architectures
small_graphrag = evaluate_query(small_example['side_effect'], graphrag, 'GraphRAG')
small_format_b = evaluate_query(small_example['side_effect'], format_b, 'Format_B_Chunked')
small_format_a = evaluate_query(small_example['side_effect'], format_a, 'Format_A')

# Compare
print("\n" + "="*80)
print(f"SMALL TIER COMPARISON: {small_example['side_effect']}")
print("="*80)
print(f"{'Architecture':<20} {'Recall':<12} {'Precision':<12} {'Latency':<12}")
print("-"*80)

for res in [small_graphrag, small_format_b, small_format_a]:
    print(f"{res['architecture']:<20} {res['recall']*100:<11.2f}% {res['precision']*100:<11.2f}% {res['latency']:<11.2f}s")

print("="*80)

## Example 4: Rare Tier Query (5-19 drugs)

In [ ]:
# Pick first rare tier example
rare_example = tier_queries['rare'][0]
print(f"Testing: {rare_example['side_effect']} (tier: {rare_example['tier']})")
print(f"Expected drugs: {len(ground_truth[rare_example['side_effect']])}")

In [ ]:
# Run all three architectures
rare_graphrag = evaluate_query(rare_example['side_effect'], graphrag, 'GraphRAG')
rare_format_b = evaluate_query(rare_example['side_effect'], format_b, 'Format_B_Chunked')
rare_format_a = evaluate_query(rare_example['side_effect'], format_a, 'Format_A')

# Compare
print("\n" + "="*80)
print(f"RARE TIER COMPARISON: {rare_example['side_effect']}")
print("="*80)
print(f"{'Architecture':<20} {'Recall':<12} {'Precision':<12} {'Latency':<12}")
print("-"*80)

for res in [rare_graphrag, rare_format_b, rare_format_a]:
    print(f"{res['architecture']:<20} {res['recall']*100:<11.2f}% {res['precision']*100:<11.2f}% {res['latency']:<11.2f}s")

print("="*80)

## Summary: Aggregate Statistics Across Tiers

In [ ]:
# Collect all results
all_results = [
    # Large
    (result_graphrag, result_format_b, result_format_a),
    # Medium
    (medium_graphrag, medium_format_b, medium_format_a),
    # Small
    (small_graphrag, small_format_b, small_format_a),
    # Rare
    (rare_graphrag, rare_format_b, rare_format_a)
]

print("\n" + "="*80)
print("SUMMARY: 4 Sample Queries Across All Tiers")
print("="*80)

# Calculate averages for each architecture
architectures_data = {
    'GraphRAG': [],
    'Format_B_Chunked': [],
    'Format_A': []
}

for tier_results in all_results:
    for res in tier_results:
        architectures_data[res['architecture']].append(res)

print(f"\n{'Architecture':<20} {'Avg Recall':<12} {'Avg Precision':<15} {'Avg Latency':<12}")
print("-"*80)

for arch_name, results in architectures_data.items():
    avg_recall = sum(r['recall'] for r in results) / len(results)
    avg_precision = sum(r['precision'] for r in results) / len(results)
    avg_latency = sum(r['latency'] for r in results) / len(results)
    
    print(f"{arch_name:<20} {avg_recall*100:<11.2f}% {avg_precision*100:<14.2f}% {avg_latency:<11.2f}s")

print("="*80)

# Speed comparison
graphrag_avg_latency = sum(r['latency'] for r in architectures_data['GraphRAG']) / 4
format_b_avg_latency = sum(r['latency'] for r in architectures_data['Format_B_Chunked']) / 4

print(f"\n💡 Key Findings from Sample Queries:")
print(f"   • GraphRAG: {sum(r['recall'] for r in architectures_data['GraphRAG'])/4*100:.1f}% avg recall")
print(f"   • Format_B: {sum(r['recall'] for r in architectures_data['Format_B_Chunked'])/4*100:.1f}% avg recall")
print(f"   • Format_A: {sum(r['recall'] for r in architectures_data['Format_A'])/4*100:.1f}% avg recall")
print(f"\n   • GraphRAG is {format_b_avg_latency/graphrag_avg_latency:.0f}× faster than Format_B")
print(f"   • GraphRAG achieves perfect accuracy with minimal latency")

## Visualization: Performance Comparison

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Prepare data
tiers = ['Large', 'Medium', 'Small', 'Rare']

graphrag_recalls = [result_graphrag['recall'], medium_graphrag['recall'], 
                   small_graphrag['recall'], rare_graphrag['recall']]
format_b_recalls = [result_format_b['recall'], medium_format_b['recall'], 
                   small_format_b['recall'], rare_format_b['recall']]
format_a_recalls = [result_format_a['recall'], medium_format_a['recall'], 
                   small_format_a['recall'], rare_format_a['recall']]

graphrag_latencies = [result_graphrag['latency'], medium_graphrag['latency'], 
                     small_graphrag['latency'], rare_graphrag['latency']]
format_b_latencies = [result_format_b['latency'], medium_format_b['latency'], 
                     small_format_b['latency'], rare_format_b['latency']]
format_a_latencies = [result_format_a['latency'], medium_format_a['latency'], 
                     small_format_a['latency'], rare_format_a['latency']]

# Create figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Recall by Tier
x = np.arange(len(tiers))
width = 0.25

ax1.bar(x - width, [r*100 for r in graphrag_recalls], width, label='GraphRAG', color='green', alpha=0.8)
ax1.bar(x, [r*100 for r in format_b_recalls], width, label='Format_B_Chunked', color='blue', alpha=0.8)
ax1.bar(x + width, [r*100 for r in format_a_recalls], width, label='Format_A', color='red', alpha=0.8)

ax1.set_xlabel('Tier')
ax1.set_ylabel('Recall (%)')
ax1.set_title('Recall by Tier')
ax1.set_xticks(x)
ax1.set_xticklabels(tiers)
ax1.legend()
ax1.set_ylim([0, 105])
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Latency by Tier (log scale)
ax2.bar(x - width, graphrag_latencies, width, label='GraphRAG', color='green', alpha=0.8)
ax2.bar(x, format_b_latencies, width, label='Format_B_Chunked', color='blue', alpha=0.8)
ax2.bar(x + width, format_a_latencies, width, label='Format_A', color='red', alpha=0.8)

ax2.set_xlabel('Tier')
ax2.set_ylabel('Latency (seconds, log scale)')
ax2.set_title('Latency by Tier')
ax2.set_xticks(x)
ax2.set_xticklabels(tiers)
ax2.legend()
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('benchmark_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Chart saved as: benchmark_comparison.png")

## Detailed Example: Inspect GraphRAG Query Execution

In [ ]:
# Let's look at what GraphRAG actually returns
side_effect = "headache"
print(f"Query: What drugs cause '{side_effect}'?\n")

result = graphrag.reverse_query(side_effect)

print(f"Result structure:")
print(f"  Keys: {list(result.keys())}")
print(f"  Number of drugs: {len(result.get('drugs', []))}")
print(f"\nFirst 10 drugs:")
for i, drug in enumerate(result.get('drugs', [])[:10], 1):
    print(f"  {i}. {drug}")

# Verify against ground truth
expected = set([d.lower() for d in ground_truth.get(side_effect, [])])
extracted = set([d.lower() for d in result.get('drugs', [])])

print(f"\nValidation:")
print(f"  Expected: {len(expected)} drugs")
print(f"  Extracted: {len(extracted)} drugs")
print(f"  Match: {len(expected & extracted)} drugs")
print(f"  Recall: {len(expected & extracted) / len(expected) * 100:.2f}%")

## Production-Ready Example: Batch Queries

In [ ]:
# Simulate a batch of user queries
batch_queries = [
    "headache",
    "nausea",
    "dizziness",
    "fatigue",
    "insomnia"
]

print("Running batch of 5 queries with GraphRAG...\n")

batch_start = time.time()
batch_results = []

for se in batch_queries:
    start = time.time()
    result = graphrag.reverse_query(se)
    latency = time.time() - start
    
    batch_results.append({
        'side_effect': se,
        'drug_count': len(result.get('drugs', [])),
        'latency': latency
    })
    
    print(f"  {se:15} → {len(result.get('drugs', [])):4} drugs in {latency:.3f}s")

total_time = time.time() - batch_start

print(f"\nBatch completed in {total_time:.2f}s")
print(f"Throughput: {len(batch_queries) / total_time:.2f} queries/second")
print(f"Avg latency: {sum(r['latency'] for r in batch_results) / len(batch_results):.3f}s")
print(f"\n✅ Ready for production deployment!")

## Conclusion

**Key Takeaways:**

1. **GraphRAG is the clear winner** for reverse queries on structured data
   - 100% accuracy across all tiers
   - Sub-second latency (0.09s average)
   - 914× faster than Format_B_Chunked
   - Production-ready throughput (11+ queries/second)

2. **Format_B_Chunked** is accurate but slow
   - 98.88% recall (near-perfect)
   - 82.44s average latency (not suitable for real-time)
   - Valuable for research and novel discovery

3. **Format_A** is not suitable for reverse queries
   - Only 7.97% recall
   - Designed for binary classification, not reverse lookup

**Production Recommendation:** Deploy GraphRAG for all reverse query operations.